In [27]:
import cudaq
import numpy as np
from functools import reduce

In [28]:
qubit_num=2

@cudaq.kernel
def psi(num:int):
    q=cudaq.qvector(num)
    h(q[1])

psi_state=cudaq.get_state(psi,qubit_num)
print(psi_state)

0.707107+0j 0+0j 0.707107+0j 0+0j 



In [29]:
@cudaq.kernel
def phi(n:int):
    q=cudaq.qvector(n)
    x(q[0])

phi_state=cudaq.get_state(phi,qubit_num)
print(phi_state)
    

0+0j 1+0j 0+0j 0+0j 



In [30]:
ham=cudaq.spin.x(0)*cudaq.spin.x(1)
ham_matrx=ham.to_matrix()
print(ham_matrx)

(0,0) (0,0) (0,0) (1,0)
(0,0) (0,0) (1,0) (0,0)
(0,0) (1,0) (0,0) (0,0)
(1,0) (0,0) (0,0) (0,0)



In [31]:
exp_val=reduce(np.dot,(np.array(psi_state).conj().T, ham_matrx, phi_state))
print(exp_val) 

(0.7071067690849304+0j)


In [49]:
@cudaq.kernel
def U_psi(q:cudaq.qview):
    h(q[1])

@cudaq.kernel
def U_phi(q:cudaq.qview):
    x(q[0])

@cudaq.kernel  
def ham_cir(q:cudaq.qview):
    x(q[0])
    x(q[1])

@cudaq.kernel
def kernel(n:int):
    ancilla=cudaq.qubit()
    q=cudaq.qvector(n)
    h(ancilla)
    cudaq.control(U_phi,ancilla,q)
    cudaq.control(ham_cir,ancilla,q)
    cudaq.control(U_psi,ancilla,q)
    
    h(ancilla)
    
    mz(ancilla)
    
shots=10000    
count=cudaq.sample(kernel,qubit_num, shots_count=shots)    
print(count)

mean_val=(count['0']-count['1'])/shots
error=np.sqrt(2*count['0']*count['1']/shots)/shots
print('Observable: ', mean_val,'+ -', error)
print('numerically', np.real(exp_val))



{ 0:8512 1:1488 }

Observable:  0.7024 + - 0.005033061891135455
numerically 0.7071067690849304
